# The battle of neighbourhoods

In this Week im goint to check the diferences between the neighbourhoods of the East Toronto Borough, trying to understand what kind of business are well rated in each neighbourhoods and helping a business man that would like to know where to open a new restaurant and what kind of restaurant should it open in East Toronto Borough

In order to do that, we are going to use the dataset from the previous week, filtering the final dataframe by East Toronto.
After that we are going to get some information about the shops in that area trying to find a market niche to deploy a new shop
with more odds of success

The following code lines are for showing how i achieved the main data set for this proposal

In [3]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import xml

import json
import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



Getting the wikipedi data and finding the scope fields by tagging the html
note that im using the old wikipedia ID

In [4]:
url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text
soup = BeautifulSoup(url,'lxml')

In [5]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Deleting Not assigned Borough & Grouping Neighbourhoods and joining in a simple field separated by ,

In [6]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df["Neighbourhood"]=np.where(df['Neighbourhood']=='Not assigned', df['Borough'],df['Neighbourhood'])

# Getting latitude & longitude

In [8]:
df_g = pd.read_csv('http://cocl.us/Geospatial_data')
df_g.columns = ['Postcode', 'Latitude', 'Longitude']

join the two dataframes by Postcode Field

In [9]:
df_p = pd.merge(df, df_g, on=['Postcode'], how='inner')

df_Toronto = df_p[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_Toronto.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Selecting only East Toronto values

In [11]:
df_t = df_Toronto[df_Toronto['Borough']=='East Toronto']

toronto = df_t.reset_index(drop=True)
toronto

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558


from here in advance will be explained en the week 2 of The Battle of neighbourhoods